Exact geometric predicates
Examples from 

ESBMC is working with C++ https://ssvlab.github.io/esbmc/documentation.html
also python apparently?
Try my own version of AI thing


Checkin

Floats. Or fixed point.

Comparing two programs



# Kani

https://model-checking.github.io/kani-verifier-blog/

https://model-checking.github.io/kani/install-guide.html
! cargo install kani-verifier

kani --visualize

```
#[kani::proof]
#[kani::should_panic]
#[kani::unwind(<number>)]
#[kani::solver(<solver>)]
#[kani::stub(<original>, <replacement>)]
```

https://model-checking.github.io/kani/tool-comparison.html  

https://github.com/tokio-rs/loom  https://github.com/awslabs/shuttle concurrency testing tools

In [3]:
%%file /tmp/kanitest.rs
fn estimate_size(x: u32) -> u32 {
    if x < 256 {
        if x < 128 {
            return 1;
        } else {
            return 3;
        }
    } else if x < 1024 {
        if x > 1022 {
            panic!("Oh no, a failing corner case!");
        } else {
            return 5;
        }
    } else {
        if x < 2048 {
            return 7;
        } else {
            return 9;
        }
    }
}

#[cfg(kani)]
#[kani::proof]
fn check_estimate_size() {
    let x: u32 = kani::any();
    estimate_size(x);
}

Overwriting /tmp/kanitest.rs


In [5]:
! kani /tmp/kanitest.rs

Kani Rust Verifier 0.53.0 (standalone)
             - caller_location (1)
             - foreign function (1)
         
         Verification will fail if one or more of these constructs is reachable.
         See https://model-checking.github.io/kani/rust-feature-support.html for more details.


Checking harness check_estimate_size...
CBMC 5.95.1 (cbmc-5.95.1)
CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Reading GOTO program from file /tmp/kanitest__RNvCsc693Q3hSBuS_8kanitest19check_estimate_size.out
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 16 object bits, 48 offset bits (user-specified)
Starting Bounded Model Checking
aborting path on assume(false) at file /tmp/kanitest.rs line 10 column 13 function estimate_size thread 0
Runtime Symex: 0.0163615s
size of program expression: 1401 steps
slicing removed 821 assignments
Generated 1 VCC(s), 1 remaining after simplifica

# Goto Instrument
--dump-c
--horn

coutn effective lines of code? what does that mean
--dot
--interpreter

--isr interrupts
--mmio  memory-mapped io

In [1]:
! goto-instrument --help


* *       Goto-Instrument 5.95.1 (cbmc-5.95.1) 64-bit       * *
* *                 Copyright (C) 2008-2013                 * *
* *                     Daniel Kroening                     * *
* *                  kroening@kroening.com                  * *

Usage:                       Purpose:

 goto-instrument [-?] [-h] [--help]  show this help
 goto-instrument --version    show version and exit
 goto-instrument [options] in [out]  perform analysis or instrumentation

Dump Source:
 --dump-c                     generate C source
 --dump-c-type-header m       generate a C header for types local in m
 --dump-cpp                   generate C++ source
 --no-system-headers          generate C source expanding libc includes
 --use-all-headers            generate C source with all includes
 --harness                    include input generator in output
 --horn                       print program as constrained horn clauses

Diagnosis:
 --show-properties            show the properties, but do

# Float

# Floating Point

# Other C tooling
Frama-C

What I have in the first tutorial is pretty good.
Download cbmc 6

Look into floating point stuff at the same time?
Simpler?
PID
Kalman
Kepler / Baylonian

https://haslab.github.io/MFES/2122/CBMCexamples-handout.pdf

This is an interesting file. They replace assume with forms of __builtin_unreachable. What does gcc do with that? Does that enhance optimizations?
https://github.com/awslabs/aws-c-common/blob/main/include/aws/common/assert.h 
```


```

cross product.
abs
add
rotation
quaternion

knuckledragger?

halide?
Something image?



In [ ]:
%%file /tmp/vec2.c

#include <assert.h>
#include <math.h>
float dot_f(float[2] a, float[2]b) {
    return a[0]*b[0] + a[1]*b[1];
}

double dot_d(double[2] a, double[2]b) {
    return a[0]*b[0] + a[1]*b[1];
}

int main(){
    float a[2],b[2];
    double a1[2] = {a[0], a[1]};
    double b1[2] = {b[0], b[1]};
    float res1 = dot_f(a,b);
    double res2 = dot_d(a1,b1);
    assert(res1 >= res2 - 1e-6 && res1 <= res2 + 1e-6); 
    return 0;
}

## Kalman

Kalman filters are a methdology for estimating state in the presence of noisy measurements and noisy dynamics.

You can take an optimization function / curve fitting perspective or a bayesian persepctive.

The curve fitting persepctive is to optimize a state that best fits the prior know dynamics and observations


There are both continuous and discrete time kalman filters.

There are also extdned kalman filters which locally and iteratively approximate the dynamics with linear approximations.

A ball and spring
$\dot{x} = v$
$m \dot{v} = -x$

$ x_{t+1} = Ax_{t} + \epsilon$
$ o_{t} = Bx + \delta$

$ f(x_{t+1} - (Ax_{t} + \epsilon ) ) + g(o_{t} - Bx + \delta) + h(\epsilon) + k(\delta)$

$ f(x) = x^2$

Or flip the dynamics into a constraint, whatever floats your boat. A common method of solving constraints is the barrier method though, so it all is kind of the same thing.


The online character.
Solving these equations involves linear algebra. But the next timestep isn't that different of a problem.
This csan be done as a low rank update.


What is the point of verifying a kalman filter?

- The reals are a delusion. Since the parameters of te kalman filter are measured or inferred by tweaking and tuning, the exact mathemtical model isn't that relevant. It is concrete and well defined though, which is tantalizing as a verifier stunt. 
- We really don't want runtime errors liek div 0 or loop windup or overflow. Those are alarming and bad
- We do sometimes tune and test something at one precision and want to switch to another kind of precision for all sorts of reasons. Or similarly, swap out the CPU or move to/from an fpga. Confirming that the logical behavior is identical to whatever it was before is a failry well defined question and a useful one.
- It is good to know the robot never goes into the lava (unless it's a lava robot, in which case its good to know it does go into the lava).


Tabina verified radart imaging

PID controllers.

PID controllers are perversely effective.


https://github.com/search?q=kalman&type=repositories
https://github.com/TKJElectronics/KalmanFilter http://blog.tkjelectronics.dk/2012/09/a-practical-approach-to-kalman-filter-and-how-to-implement-it
https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python
https://github.com/mherb/kalman

## Frama C


In [ ]:
%%bash
opam install frama-c
why3 config detect

I consider Frama-C to be the big dog in the room when it comes to C verification.

mlcfg + pcode?

https://frama-c.com/2022/11/16/supertest-solidsands.html This was already the case with NIST’s Juliet C/C++ testsuite, [J. Moerman’s 2018 bachelor’s thesis test suite (JM2018TS)(https://github.com/JMoerman/JM2018TS), TrailOfBits’ version of the DARPA Challenge Binaries, 

https://en.wikipedia.org/wiki/Frama-C

- eva evolved value analysis https://frama-c.com/fc-plugins/eva.html
- wp
- jessie - old. kind of replaced by wp?
- e-acsl - adds runtime assertions
- Pathcraewler - https://frama-c.com/fc-plugins/pathcrawler.html generate unit tests. proprietary

- rte - runtime error detection. preprocessor to wp or e-acsl
- ltl properties https://frama-c.com/fc-plugins/aorai.html
- https://frama-c.com/fc-plugins/metrics-calculation.html metrics. cyclomatic, yuck. but eva coverage ok
- https://frama-c.com/fc-plugins/instantiate.html specialize function signatures?
- constant folding + useless code elimination. A frameowrk for partial eval?

eva is probably the most scalable thing?

invariant inference?

https://www.sciencedirect.com/science/article/pii/S0167642314001671 Verification of the functional behavior of a
floating-point program: An industrial case study

In [ ]:
%%file /tmp/quat.c
typedef double quat[4];

/// copy of a quaternion
void Quat_copy(const quat src,quat dst) {
    dst[0] = src[0];
    dst[1] = src[1];
    dst[2] = src[2];
    dst[3] = src[3];
}

/// multiplication of quaternions
void Quat_prod(const quat q1, const quat q2, quat q) {
    q[0] = q1[0]*q2[0] - q1[1]*q2[1] - q1[2]*q2[2] - q1[3]*q2[3];
    q[1] = q1[0]*q2[1] + q1[1]*q2[0] + q1[2]*q2[3] - q1[3]*q2[2];
    q[2] = q1[0]*q2[2] - q1[1]*q2[3] + q1[2]*q2[0] + q1[3]*q2[1];
    q[3] = q1[0]*q2[3] + q1[1]*q2[2] - q1[2]*q2[1] + q1[3]*q2[0];
}

/// returns a random quaternion of norm 1
void random_unit_quat(quat q);

/// repeated multiplication of quaternions of norm 1
int test1(void) {
    quat current, next, incr;
    random_unit_quat(current);
    while (1) {
        random_unit_quat(incr);
        Quat_prod(current,incr,next);
        Quat_copy(next,current);
    }
    return 0;
}

In [ ]:
%%file /tmp/frama1.c

#include <stdio.h>
int main() {
    int a = 1;
    int b = 2;
    int c = a + b;
    printf("%d\n", c);
    return 0;
}

In [ ]:
! frama-c -eva /tmp/frama1.c

In [ ]:
%%file /tmp/frama2.c
// from https://frama-c.com/fc-plugins/eva.html
int abs(int x) {
  if (x < 0) return -x;
  else return x;
}

In [ ]:
! frama-c -eva /tmp/frama2.c -main abs 

In [ ]:
%%file /tmp/ex1.c
#include <assert.h>
int main(){
    int x;
    if (x <= 42){
            assert(x != 12345);
    }
}

In [ ]:
! frama-c -eva /tmp/ex1.c

In [ ]:
! frama-c -wp -wp-rte /tmp/frama1.c

In [ ]:
! frama-c -plugins

# Worst Case Execution Time
https://www.cprover.org/cbmc/applications/


In [46]:
%%file /tmp/tick.c
#define TICK(N) (_time += N)

int _time = 0;

void frobulate();

int main() {
    TICK(1);
    TICK(2);
    for (int i = 0; i < 10; i++) {
        frobulate();
        TICK(1);
    }
    assert(_time <= 10);
}


Overwriting /tmp/tick.c


In [47]:
! cbmc /tmp/tick.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/tick.c
Converting
Type-checking tick
file /tmp/tick.c line 14 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
**** WARNING: no body for function frobulate
Unwinding loop main.0 iteration 1 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 2 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 3 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 4 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 5 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 6 file /tmp/tick.c line 10 function main thread 0
Unwinding loop main.0 iteration 7 file /tmp/tick.c line 10 f

In [40]:
%%file /tmp/vect3.cpp
#include <iostream>
#include <cmath>

class Vector3 {
public:
    float x, y, z;

    // Constructors
    Vector3() : x(0), y(0), z(0) {}
    Vector3(float x, float y, float z) : x(x), y(y), z(z) {}

    // Vector addition
    Vector3 operator+(const Vector3& other) const {
        return Vector3(x + other.x, y + other.y, z + other.z);
    }
    
};

int main(){
    Vector3 a(1, 2, 3);
    Vector3 b(4, 5, 6);
    Vector3 c = a + b;
    std::cout << c.x << " " << c.y << " " << c.z << std::endl;
    return 0;
}

Overwriting /tmp/vect3.cpp


In [41]:
!esbmc /tmp/vect3.cpp --unwind 4

ESBMC version 7.5.0 64-bit x86_64 linux
Target: 64-bit little-endian x86_64-unknown-linux with esbmclibc
Parsing /tmp/vect3.cpp
Converting


Generating GOTO Program
GOTO program creation time: 0.329s
GOTO program processing time: 0.010s
Starting Bounded Model Checking
Symex completed in: 0.004s (216 assignments)
Slicing time: 0.000s (removed 216 assignments)
Generated 54 VCC(s), 0 remaining after simplification (0 assignments)
BMC program time: 0.004s

VERIFICATION SUCCESSFUL


In [34]:
%%file /tmp/cpptest.cpp

//#include <iostream>
//#include <cmath>
#include <string>
int main() {
    const std::string name = "Ali";
    //std::cout << "hello world" << std::endl;

    return 0;
}

Overwriting /tmp/cpptest.cpp


In [36]:
!esbmc /tmp/cpptest.cpp --unwind 4

ESBMC version 7.5.0 64-bit x86_64 linux
Target: 64-bit little-endian x86_64-unknown-linux with esbmclibc
Parsing /tmp/cpptest.cpp
In file included from /tmp/cpptest.cpp:4:
In file included from /tmp/esbmc-cpp-headers-a0c3-e456-8315/string:8:
In file included from /tmp/esbmc-cpp-headers-a0c3-e456-8315/stdexcept:5:
/tmp/esbmc-cpp-headers-a0c3-e456-8315/exception:69:1: warning: function declared 'noreturn' should not return [-Winvalid-noreturn]
}
^
/tmp/cpptest.cpp:6:30: warning: conversion from string literal to 'char *' is deprecated [-Wc++11-compat-deprecated-writable-strings]
    const std::string name = "Ali";
                             ^
Converting


Generating GOTO Program
GOTO program creation time: 0.394s
GOTO program processing time: 0.010s
Starting Bounded Model Checking
Unwinding loop 7 iteration 1   file string.c line 78 column 3 function strlen
Unwinding loop 7 iteration 2   file string.c line 78 column 3 function strlen
Unwinding loop 7 iteration 3   file string.c line 78 column 3 function strlen
Unwinding loop 2 iteration 1   file string.c line 30 column 3 function strcpy
Unwinding loop 2 iteration 2   file string.c line 30 column 3 function strcpy
Unwinding loop 2 iteration 3   file string.c line 30 column 3 function strcpy
Not unwinding loop 2 iteration 4   file string.c line 30 column 3 function strcpy
Symex completed in: 0.004s (169 assignments)
Slicing time: 0.002s (removed 127 assignments)
Generated 42 VCC(s), 18 remaining after simplification (42 assignments)
No solver specified; defaulting to Boolector
Encoding remaining VCC(s) using bit-vector/floating-point arithmetic
Encoding to solver time: 0.000s
Solving with

---
layout: post
title: C Bounded Model Checker 2: Loops, Contracts, Equivalence checking
---

## CBMC and Ghidra

This could be it's own post.

ASM-BMC





## Loops

Ok, we've painted a rosy picture so far. Obviously CBMC does not scale to arbitrarily large and difficult problems

The basic technique is to unwind them. All is not lost, many loops (for loops in particular) can be completely unwound. You can add `--unwinding-assertions` to know if you've covered all possible executions. Ever if you can't, passing these chekcs does give you some confidence.

If you're going to the next level there is also the ability to add invariant annotations. I'm not sure if CBMC can infer these successfully

On bigger problems, I've had some success fiddling with the 


Another interesting thing to note is that CBMC is a limitted termination checker. If your function has a bound on the number of iterations, `--unwinding-assertions` can determine that there is nothing that crosses the bound. This may be usefulfor checking uppper bounds of possible loop iterations which can matter in realtime contexts. Another thing this could be useful for is checking eBPF programs, which are required to have a determinstic bound, usually enforced by requiring an acyclic control flow graph.


https://diffblue.github.io/cbmc/cprover-manual/md_cbmc_tutorial.html

https://diffblue.github.io/cbmc//contracts-loops.html
https://diffblue.github.io/cbmc//contracts-loop-invariants.html Documentation on loop invariants


In [34]:
%%file /tmp/sum.c
#include <stdint.h>
int sum(uint32_t n)
    __CPROVER_requires(n <= 3)
    {
    uint32_t acc = 0;
    for(uint32_t i = 0; i < n; i++)
    __CPROVER_loop_invariant(2 * acc == i * (i - 1))
    __CPROVER_decreases(n - i)
    {
        acc += i;
    }
    return acc;
}

Overwriting /tmp/sum.c


In [35]:
%%bash
goto-cc -o /tmp/sum.goto /tmp/sum.c
goto-instrument --apply-loop-contracts /tmp/sum.goto /tmp/sum_inst.goto
cbmc /tmp/sum_inst.goto --unwinding-assertions --function sum

Reading GOTO program from '/tmp/sum.goto'
Function Pointer Removal
Virtual function removal
Cleaning inline assembler statements
Adding nondeterministic initialization of static/global variables.
Writing GOTO program to '/tmp/sum_inst.goto'
CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Reading GOTO program from file /tmp/sum_inst.goto
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
aborting path on assume(false) at file /tmp/sum.c line 6 function sum thread 0
Runtime Symex: 0.00300661s
size of program expression: 123 steps
simple slicing removed 5 assignments
Generated 10 VCC(s), 3 remaining after simplification
Runtime Postprocess Equation: 1.2704e-05s
Passing problem to propositional reduction
converting SSA


Runtime Convert SSA: 0.00571386s
Running propositional reduction
Post-processing
Runtime Post-process: 3.968e-06s
Error while terminating subprocess (pid=142076): 


bash: line 3: 142090 Killed                  cbmc /tmp/sum_inst.goto --unwinding-assertions --function sum


In [ ]:
!cbmc /tmp/sum.c --function sum

## Contracts
https://diffblue.github.io/cbmc/contracts-user.html

# Comparative Checking
Sum forward is the same as sum backwards

In [24]:
%%file /tmp/sum.c

extern int array[10];
 
int sum()
{
  unsigned i, sum;
 
  sum = 0;
  for(i = 0; i < 10; i++)
    sum += array[i];
 
  return sum;
}

int sum_back()
{
  int i, sum; // Interesting. i had a bug when these were unsgined. 
  // Of course the for loop does something weird in that case
 
  sum = 0;
  for(i = 9; i >= 0; i--)
    sum += array[i];
  return sum;
}

int main(){
  assert(sum() == sum_back());
}

Overwriting /tmp/sum.c


In [28]:
! cbmc /tmp/sum.c --boolector # Sat solver is super slow on this one

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/sum.c
Converting
Type-checking sum
file /tmp/sum.c line 26 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Unwinding loop sum.0 iteration 1 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 2 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 3 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 4 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 5 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 6 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 7 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 8 file /tmp/sum.c line 

A very interesting topic is that of security properties / information leakage. how do you write specs for this? What is a definition of information leakage?

In [29]:
%%file /tmp/leak.c
#include <assert.h>
int safeprog(int low, int high){
    int foo = low ^ high;
    foo = ((foo << 1) ^ high) >> 1;
    return foo;
}

int main(){
    int high = nondet_int();
    int high1 = nondet_int();
    int low = nondet_int();
    //int high, high1, low;

    assert(safeprog(low,high) == safeprog(low,high1));
    return 0;
}

Overwriting /tmp/leak.c


In [30]:
!cbmc /tmp/leak.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/leak.c
Converting
Type-checking leak
file /tmp/leak.c line 9 function main: function 'nondet_int' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Runtime Symex: 0.000956779s
size of program expression: 54 steps
simple slicing removed 5 assignments
Generated 1 VCC(s), 1 remaining after simplification
Runtime Postprocess Equation: 1.647e-05s
Passing problem to propositional reduction
converting SSA
Runtime Convert SSA: 0.000395847s
Running propositional reduction
Post-processing
Runtime Post-process: 3.34e-06s
Solving with MiniSAT 2.2.1 with simplifier
703 variables, 921 clauses
SAT checker: instance is SATISFIABLE
Runtime Solver: 0.00037143s
Runtime decision procedure: 0.000804985s

** Results:
/tmp/leak.c function main
[main.asser

## C as a Specification Language
If you avoid pointer-y things, C isn't the craziest language in the world to specify things like state machines. Obviously it is non ideal for many reasons, but the extensive tooling, fast compilation, and most importantly acceptance of the general programming populace is worth a lot.

For example, TLA+ can be used to build a model of your distributed system.



In [ ]:
%file /tmp/diehard.c

int good_state(){
    
}

enum state_t {};
int main(){
    river fox = 0;
    river chicken = 0;
    river grain = 0;
    for(;;){
        switch(state){

        }
    }
}



In [ ]:
%%file /tmp/server.c



int main(){
    dispatch:
    action = non_det();
    
    action1:

    goto dispatch;
    
    action2:

    goto dispatch;
    
    action3:

    goto dispatch;
}



# Bits and Bobbles

Here's a list of serious C projects with CBMC instrumentation, many from Amazon: https://model-checking.github.io/cbmc-training/projects.html. They are playing a similar role to unit tests for library functions.

SV Comp  https://sv-comp.sosy-lab.org/ - CPAchecker UAutomizer are very successful in the competition. I have not used them as much

Klee, symcc are symbolic executors. Similar in many respects to a bounded model checker. I think the biggest philosophical difference is they aren't really centered around ensuring bug absence, instead around bug finding, which is a little different.

Frama-C
VST
https://github.com/verifast/verifast graham seemed to like this one


[esbmc](http://esbmc.org/) is a C bounded model checker.

Comparing a C function to it's grammar?

```bash
echo "
#include<stdbool.h>

bool check_balance(char *input){
    int count = 0;
    while(*input != '\0'){
        if(*input == '(') count++;
        if(*input == ')') count--;
        input++;
    }
    return count == 0;
" > /tmp/parens.c
cbmc /tmp/parens.c 
```


On my last post, I got a lot of guff for having undefined behavior. CBMC is not great for detecting this stuff. Ok, so what can I use?

Indeed `-Wall -Wpedantic` notices. Can I trick it?

In [35]:
%%file /tmp/ub.c
#include <assert.h>
int main(){
    int x;
    x++;
    assert(x != 12345);
}

Overwriting /tmp/ub.c


In [38]:
! clang -O1 -Wall -Wpedantic -fsanitize=undefined /tmp/ub.c -o /tmp/ub.out && /tmp/ub.out

/tmp/ub.c:2:9: warning: a function declaration without a prototype is deprecated in all versions of C [-Wstrict-prototypes]
    2 | int main(){
      |         ^
      |          void
/tmp/ub.c:4:5: warning: variable 'x' is uninitialized when used here [-Wuninitialized]
    4 |     x++;
      |     ^
/tmp/ub.c:3:10: note: initialize the variable 'x' to silence this warning
    3 |     int x;
      |          ^
      |           = 0
2 warnings generated.


In [40]:
! objdump -d /tmp/ub.out | grep -A 10 main.: # This is a disassembly of the main function of the compiled program.
 

000000000002ea28 <main>:
   2ea28:	31 c0                	xor    %eax,%eax
   2ea2a:	c3                   	ret    
   2ea2b:	0f 1f 44 00 00       	nopl   0x0(%rax,%rax,1)

000000000002ea30 <atexit>:
   2ea30:	f3 0f 1e fa          	endbr64 
   2ea34:	48 8b 15 3d 57 01 00 	mov    0x1573d(%rip),%rdx        # 44178 <__dso_handle>
   2ea3b:	31 f6                	xor    %esi,%esi
   2ea3d:	e9 ae 66 fd ff       	jmp    50f0 <__cxa_atexit@plt>



There's nothing there. Indeed, the compiler fused it all out.

In [10]:
%%file /tmp/foo.c
int mystrcmp(const char *cs, const char *ct) {
  unsigned char c1, c2;
  while (1) {
    c1 = *(cs++);
    c2 = *(ct++);
    if (c1 != c2)
      return c1 < c2 ? -1 : 1;
    if (!c1)
      break;
  }
  return 0;
}

int foo(char **argv) {
  char *command = argv[1];
  //assert(mystrcmp("STORE", command) != 0);
  assert(strcmp("STORE", command) != 0);
  return 0;
}

Overwriting /tmp/foo.c


In [5]:
!cbmc --help


* *             CBMC 5.95.1 (cbmc-5.95.1) 64-bit            * *
* *                 Copyright (C) 2001-2018                 * *
* *              Daniel Kroening, Edmund Clarke             * *
* * Carnegie Mellon University, Computer Science Department * *
* *                  kroening@kroening.com                  * *
* *        Protected in part by U.S. patent 7,225,417       * *

Usage:                       Purpose:

 cbmc [-?] [-h] [--help]      show this help
 cbmc --version               show version and exit
 cbmc [options] file.c ...    perform bounded model checking

Analysis options:
 --show-properties            show the properties, but don't run analysis
 --symex-coverage-report f    generate a Cobertura XML coverage report in f
 --property id                only check one specific property
 --trace                      give a counterexample trace for failed properties
 --stop-on-fail               stop analysis once a failed property is detected 
                         

In [14]:
!cbmc --function foo /tmp/foo.c --trace

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/foo.c
Converting
Type-checking foo
file /tmp/foo.c line 17 function foo: function 'strcmp' is not declared
file /tmp/foo.c line 17 function foo: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
file <builtin-library-strcmp> line 9: warning: implicit function declaration "strcmp"
old definition in module foo file /tmp/foo.c line 17 function foo
signed int (void)
new definition in module <built-in-library> file <builtin-library-strcmp> line 9
signed int (const char *s1, const char *s2)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Unwinding loop strcmp.0 iteration 1 file <builtin-library-strcmp> line 44 function strcmp thread 0
Unwinding loop strcmp.0 iteration 2 file <builtin-library-strcmp> line 44 function strcmp thread 0
Unwinding loop strcmp.0 iteration 3 

In [9]:
!cbmc /tmp/foo.c --show-goto-functions

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/foo.c
Converting
Type-checking foo
file /tmp/foo.c line 16 function foo: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

foo /* foo */
        // 29 file /tmp/foo.c line 15 function foo
        DECL foo::1::command : signedbv[8]*
        // 30 file /tmp/foo.c line 15 function foo
        ASSIGN foo::1::command := *(foo::argv + cast(1, signedbv[64]))
        // 31 file /tmp/foo.c line 16 function foo
        DECL foo::$tmp::return_value_mystrcmp : signedbv[32]
        // 32 file /tmp/foo.c line 16 function foo
        CALL foo::$tmp::return_value_mystrcmp := mystrcmp(address_of("STORE"[0]), foo::1::command)
        // 33 file /tmp/foo.c line 16 function foo
        ASSERT foo::$tmp::return_value_mystrcmp ≠ 0 // assertion return_value_mystrcmp != 0
    

Nathan Chong, Byron Cook, Konstantinos Kallas, Kareem Khazem, Felipe R. Monteiro, Daniel Schwartz-Narbonne, Serdar Tasiran, Michael Tautschnig, and Mark R. Tuttle. 2020. Code-level model checking in the software development workflow. In Proceedings of the ACM/IEEE 42nd International Conference on Software Engineering: Software Engineering in Practice (ICSE-SEIP '20). Association for Computing Machinery, New York, NY, USA, 11–20. https://doi.org/10.1145/3377813.3381347

https://github.com/awslabs/aws-c-common




# Formal Methods
-
-


# CBMC
- AWS
- paper
- bootloader

# Selected applications
CBMC bullet points

C Bounded Model Checker
Symbolic execution of C code
Highly Automated
Does not scale to one shot verification of large codebases. This can be attacked by breaking up the problem manually into “proof harnesses”.
Finds many problems automatically. Int overflow, bounds violations, use after free
https://news.ycombinator.com/item?id=39191507 The C bounded model checker: criminally underused . My introductory blog post
An extremely useful intuitive tool to ask nearly any question about moderately sized chunks of C code.
AWS is leaning hard into CBMC. They have a methodology paper. This is not about the nuts and bolts of the theory or working of the tool, but how an organization should use it. Code Level Model checking in the software development workflow. https://dl.acm.org/doi/10.1145/3377813.3381347 10min talk https://www.youtube.com/watch?v=-EuIrAP9_tU blog post https://www.amazon.science/blog/how-to-integrate-formal-proofs-into-software-development https://github.com/awslabs/aws-c-common
make specifications explicit in the code
write unit-test like proofs in declarative style
integrate proof artifacts into the developer work-flow. In repo.
fix bugs instead of just reporting them

pull requests review of specifications
Specifications are code. No “two language problem” for spec language

“Infinite” unit tests
More complete analysis than analyzers. 
Less complete than interactive verification like Frama-C, Coq VST. Way less human effort also. 
Not a golden bullet. Defense in depth
Testing is still necessary. Validating model with respect to the world
Property Based Testing/Fuzzing is the next tier up that can find unexpected things
Undefined Behavior not well detected currently https://github.com/diffblue/cbmc/issues/7732
Clang -Wall -Wextra -fanalyze can catch other useful bugs
https://codechecker.readthedocs.io/en/latest/ can run many analyses in CI
Frama-C has a number of useful automated analyses. Eva value analysis plugin


Industrial Use Cases https://www.cprover.org/cbmc/applications/:
Simulink equivalence checking
Partial Translation Verification for Untrusted Code-Generators CBMC is used for translation validation of Simulink and generated ANSI-C
. Translation Validation for Stateflow to C CBMC is used for translation validation of Simulink and generated ANSI-C.
Benchmarking Software Model Checkers on Automotive Code CBMC is applied to two R&D prototype Simulink models from Ford
Test Vector Generation. CBMC can generate test sets to exercise every code path
Worst Case Execution Time – #define TIC(N) {_time += N;} 
 Scalable and precise estimation and debugging of the worst-case execution time for analysis-friendly processors: a comeback of model checking
 CBMC is used for WCET analysis of programs for the Atmel AVR family
WCET Overapproximation for Software in the Context of a Cyber-Physical System
 CBMC is used to overapproximate the WCET of SPARC code in a CPS controller.
Scalable and Precise Refinement of Cache Timing Analysis via Model Checking
 An application of CBMC to WCET with emphasis on cache performance.
Cyper-physical Systems and Control
Robustness Analysis of Floating-Point Programs by Self-Composition
A Verified Low-Level Implementation of the Adaptive Exterior Light and Speed Control System
 CBMC is used to verify a MISRA-C implementation of the 2020 ABZ case study.
Experience with Static PLC Code Analysis at CERN
 CBMC is used to verify PLC code.
Cyber/Physical Co-Verification for Developing Reliable Cyber-Physical Systems
 CBMC is used to verify a combination of a controller and a plant-model in the automotive domain.
Runtime Verication for Ultra-Critical Systems
 Copilot uses CBMC to verify embedded C programs generated from a Haskell-based data-flow DSL.
Linking Functional Requirements and Software Verification
 An application of CBMC to check functional requirements of automotive software.


unit testing vs Property based testing vs model checking. Left to right is more complete coverage. More complex technique. Not necessarily that much arder though
Compared to 

# Comparison with Fuzzing / Property based Testing

- Sometimes eaiser

